In [1]:
#

In [2]:
import pandas as pd 
import os
import numpy as np 
import cv2
from keras.preprocessing import image 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from skimage.segmentation import mark_boundaries 

In [3]:
# reducing the image value under 1
def get_image_value(path, dim): 
    '''Load image and convert into arrey '''
    img = image.load_img(path, target_size = dim)
    img = image.img_to_array(img)
    return img/255

In [4]:
def get_img_array(img_paths, dim): 
    '''Get list of image path and convert into numpy array'''
    final_array = []
    from tqdm import tqdm
    for path in tqdm(img_paths):
        img = get_image_value(path, dim)
        final_array.append(img)
    final_array = np.array(final_array)  
    return final_array

In [5]:
def get_tts():
    '''This function will create a train test split'''  
   
    DIM =  (150,150)
    np.random.seed(10)        
    pistol_paths = [f'../input/guns-ar/Separated/Separated/FinalImages/Pistol/{i}' for i in os.listdir('../input/guns-ar/Separated/Separated/FinalImages/Pistol')] 
    pistol_labels = [1 for i in range(len(pistol_paths))]
    rifle_paths = [f'../input/guns-ar/Separated/Separated/FinalImages/Rifle/{i}' for i in os.listdir('../input/guns-ar/Separated/Separated/FinalImages/Rifle')] 
    rifle_labels = [2 for i in range(len(rifle_paths))]    
    neg_paths = [f'../input/guns-ar/Separated/Separated/FinalImages/NoWeapon/{i}' for i in os.listdir('../input/guns-ar/Separated/Separated/FinalImages/NoWeapon')]
    np.random.shuffle(neg_paths)
    neg_paths = neg_paths[:len(pistol_paths)- 500]
    neg_labels = [0 for i in range(len(neg_paths))]

    np.random.shuffle(pistol_paths)
    pistol_paths = pistol_paths[:len(rifle_paths)+150]
    neg_paths = neg_paths[:len(rifle_paths)+150]

    pistol_labels = [1 for i in range(len(pistol_paths))]
    rifle_labels = [2 for i in range(len(rifle_paths))]
    neg_labels = [0 for i in range(len(neg_paths))]
    paths = pistol_paths + rifle_paths + neg_paths
    labels = pistol_labels + rifle_labels + neg_labels
    x_train, x_test, y_train, y_test = train_test_split(paths, labels, stratify = labels, train_size = .90, random_state = 10)

    new_x_train = get_img_array(x_train, DIM)
    new_x_test = get_img_array(x_test, DIM)
    
    print('Train Value Counts')
    print(pd.Series(y_train).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Test Value Counts')
    print(pd.Series(y_test).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Train Shape')
    print(new_x_train.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Test Shape')
    print(new_x_test.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_test = to_categorical(y_test)
    y_train = to_categorical(y_train)
    tts = (new_x_train, new_x_test, y_train, y_test)
    return tts


In [6]:
#loading the images to varibles
x_train, x_test, y_train, y_test = get_tts()

100%|██████████| 441/441 [00:02<00:00, 153.06it/s]


Train Value Counts
0    1368
1    1368
2    1233
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test Value Counts
0    152
1    152
2    137
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Train Shape
(3969, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Test Shape
(441, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [7]:
import numpy as np 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Dense, Dropout, Flatten 
from keras.optimizers import Adam
from keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 
import matplotlib.pyplot as plt
import os
import pickle
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2
import tensorflow as tf

In [8]:
def get_conv_model(dim = (150,150, 3)):
    '''This function will create and compile a CNN given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)    
    model = Sequential() 
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3))) 
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))  
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    model.compile(loss = 'categorical_crossentropy', 
                  optimizer = optimizer, 
                  # comment experimental_steps_per_execution if not using Tpu
                  experimental_steps_per_execution = 50, 
                  metrics = ['accuracy'])
    return model 

In [9]:
#TPU Initialization

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
z = strategy.run(matmul_fn, args=(a, b))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU')]


In [10]:

#prevents overfitting and saves models every time the validation loss improves
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 2, mode = 'min')
epochs = 1000
batch_size = 32
model = get_conv_model()
model_history = model.fit(x_train, y_train, batch_size = batch_size, 
                          epochs = epochs, 
                          callbacks = [early_stopping, model_checkpoint, lr_plat], 
                          validation_data = (x_test, y_test), 
                          verbose= 1)


Epoch 1/1000
125/125 [==============================] - 21s 171ms/step - loss: 11.8002 - accuracy: 0.3726 - val_loss: 10.3867 - val_accuracy: 0.5828

Epoch 00001: val_loss improved from inf to 10.38671, saving model to ModelWeights.h5
Epoch 2/1000
125/125 [==============================] - 17s 139ms/step - loss: 9.7547 - accuracy: 0.5900 - val_loss: 8.6359 - val_accuracy: 0.6531

Epoch 00002: val_loss improved from 10.38671 to 8.63588, saving model to ModelWeights.h5
Epoch 3/1000
125/125 [==============================] - 19s 151ms/step - loss: 8.1551 - accuracy: 0.6934 - val_loss: 7.3225 - val_accuracy: 0.6984

Epoch 00003: val_loss improved from 8.63588 to 7.32249, saving model to ModelWeights.h5
Epoch 4/1000
125/125 [==============================] - 19s 151ms/step - loss: 6.9265 - accuracy: 0.7272 - val_loss: 6.1847 - val_accuracy: 0.7778

Epoch 00004: val_loss improved from 7.32249 to 6.18474, saving model to ModelWeights.h5
Epoch 5/1000
125/125 [==============================] - 